In [1]:
import pandas as pd
import altair as alt

cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## 6.7. Varrer &amp; Interligar, Revisitado

Anteriormente nesse capítulo vimos um exemplo de _varrer &amp; interligar_: usando um histograma de consulta dinâmica para destacar pontos num gráfico de dispersão de filmes. Aqui visitaremos alguns exemplos adicionais envolvendo seleções interligadas.

Voltando ao dataset `cars`, podemos usar o operador `repeat` para construir uma [Matriz de gráficos de dispersão (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que mostra associações entre quilometragem, aceleração e potência em cavalos. Podemos definir uma seleção `interval` e inclui-la _dentro_ da nossa especificação repetida de gráficos de dispersão para habilitar seleções ligadas dentre todos os gráficos.

_Clique e arraste em qualquer um dos gráficos abaixo para varrer &amp; interligar!_

In [ ]:
brush = alt.selection_interval(
    resolve='global' # resolver todas as seleções para uma única instância global
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

Note acima o uso de `resolve='global'` na seleção `interval`. A configuração padrão de `'global'` indica que dentre todos os gráficos apenas uma varredura pode estar ativa por vez. Entretanto, em alguns casos nos podemos querer definir varreduras em vários gráficos e juntar os resultados. Se usarmos `resolve='union'`, a seleção será a _união_ de todas as varreduras: se um ponto está dentro de qualquer área varrida será selecionado. Em contraste com o uso de `resolve='intersect'`, a seleção consistirá na _interseção_ de todas as varreduras: apenas pontos que estão dentro de todas as áreas varridas serão selecionados.

_Tente colocar o parâmetro `resolve` para `'union'` e para `'intersect'` e veja como se altera a lógica de seleção._

### 6.7.1. Filtragem Cruzada

Os exemplos sobre varre &amp; interligar que vimos usam codificações condicionais, por exemplo para mudar os valores da opacidade em resposta a uma seleção. Outra opção é usar a seleção definida em um gráfico para _filtrar_ o conteúdo de outro gráfico.

Vamos construir uma coleção de histogramas para o dataset `flights`: atraso `delay` (o quão cedo ou o quão tarde o voo pousou, em minutos), distância percorrida (em milhas) `distance` e horário de decolagem `time` (horário do dia). Usaremos o operador `repeat` para criar os histogramas e adicionar uma seleção `interval` para o eixo `x` com varreduras que funcionam por interseção.

Em particular, cada histograma consistirá de duas camadas: uma camada de fundo cinza e uma camada superior azul, com a camada superior filtrada pela nossa seleção de varreduras por interseção. O resultado é uma interação de _filtragem cruzada_ com os três gráficos!

_Use as varreduras arrastando nos gráficos abaixo. Ao selecionar voos com atrasos maiores ou menores, como a distribuição da distância percorrida e do horário de decolagem se comportam?_

In [ ]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # até 100 caixas
        axis=alt.Axis(format='d', titleAnchor='start') # formato inteiro, título alinhado à esquerda
    ),
    alt.Y('count():Q', title=None) # sem título no eixo y
)
  
alt.layer(
    hist.add_selection(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # atrasos de fixação > 3 horas
    time='hours(datum.date) + minutes(datum.date) / 60' # horas fracionárias
).configure_view(
    stroke='transparent' # sem esboço
)

_Usando a filtragem cruzada você pode observar que voos atrasados tem mais chance de terem decolado mais tarde no dia. Esse fenômeno é comum para viajantes frequentes: o atraso pode se propagar ao longo do dia, afetando voos subsequentes daquele mesmo avião. Para chegar na hora com chances melhores, voe cedo!_

A combinação de vários gráficos e seleções interativas pode possibilitar formas valiosas de entendimento multidimensional, tornando até histogramas básicos em aparatos de entrada poderosos para fazer perguntas sobre um dataset!

## 6.8. Resumo 

Para mais informações sobre as opções de interações suportadas pelo Altair, por favor, consulte a [Documentação de seleção interativa do Altair](https://altair-viz.github.io/user_guide/interactions.html). Para detalhes sobre costumizar gestores de eventos, por exemplo para compor várias técnicas de interação ou suportar entradas baseadas no toque em dispositivos móveis, veja a [Documentação de seleção do Vega-Lite](https://vega.github.io/vega-lite/docs/selection.html).

Quer saber mais?
- A abstração _seleção_ foi introduzida no artigo [Vega-Lite: Uma gramática de Gráficos Interativos](http://idl.cs.washington.edu/papers/vega-lite/), de Satyanarayan, Moritz, Wongsuphasawat, &amp; Heer.
- O sistema PRIM-9 (para projeções, rotações, isolamento e disfarce em até 9 dimensões) é um das ferramentas de viusalização interativa mais recentes, feita no começo dos anos 70 por Fisherkeller, Tukey, &amp; Friedman. [A retro demo video survives!](https://www.youtube.com/watch?v=B7XoW2qiFUA)
- O conceito de varrer &amp; interligar foi aperfeiçoado por Becker, Cleveland, &amp; Wilks em seu artigo de 1987 [Gráficos Dinâmicos para Análise de Dados](https://scholar.google.com/scholar?cluster=14817303117298653693).
- Para um resumo esclarecedor de técnicas de interação para visualização, veja [Dinâmica Interativa para Análise Visual](https://queue.acm.org/detail.cfm?id=2146416) de Heer &amp; Shneiderman.

### 6.8.1. Apêndice: Na Representação do Tempo

Lá atrás observamos que uma pequena protuberância no número de filmes em 1969 ou 1970. De onde vem essa protuberância? e por que 1969 _ou_ 1970? A resposta vem de uma combinação de dados faltantes e como nosso computador representa o tempo.

Internamente, datas e horários são representadas em relação à [Era Unix](https://en.wikipedia.org/wiki/Unix_time), em que o tempo "zero" corresponde à meia noite do dia 1 de Janeiro de 1970 no [fuso UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), que muda junto ao [Meriano de Greenwich](https://en.wikipedia.org/wiki/Prime_meridian). Acontece que há alguns filmes que estão com a data de lançamento faltando (`null`). Esses valores (`null`) são interpretados como tempo `0` e então mapeiam para 1 de Janeiro de 1970 no fuso UTC. Se você mora nas Américas &ndash; e, portanto, em fusos mais "cedo" &ndash; esse ponto no tempo corresponde a uma hora mais cedo no dia 31 de dezembro de 1969 no seu fuso horário. Por outro lado, se você vive próximo ou ao leste do meridiano de Greenwich, a data no seu fuso horário será dia 1 de Janeiro de 1970.

A moral? Sempre seja cético com seus dados e seja atencioso, afinal, como os dados são representados (sejam datas, números em ponto flutuante ou latitudes e longitudes, _etc._) podem, as vezes, nos levar a artefatos que impactam a análise! 